In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20201011.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,10/6/2020 19:05:29,The 400 Blows (1959),Girlhood (2014),River's Edge (1986),Adventures in Babysitting (1987),NaN,CREEPY CHILDREN FUCK OFF,OLD PEOPLE,HIGH STRESS THRILLER,THE POWER OF NATURE/THE NATURE OF POWER,THE NATURAL ENVIRONMENT: FRIEND OR FOE?,No Theme/Free-for-all
1,10/6/2020 19:23:20,River's Edge (1986),The 400 Blows (1959),Winter's Bone (2010),Adventures in Babysitting (1987),Girlhood (2014),THE POWER OF NATURE/THE NATURE OF POWER,HIGH STRESS THRILLER,THE NATURAL ENVIRONMENT: FRIEND OR FOE?,CREEPY CHILDREN FUCK OFF,OLD PEOPLE,No Theme/Free-for-all
2,10/6/2020 19:31:12,Winter's Bone (2010),River's Edge (1986),The 400 Blows (1959),Adventures in Babysitting (1987),Girlhood (2014),THE POWER OF NATURE/THE NATURE OF POWER,THE NATURAL ENVIRONMENT: FRIEND OR FOE?,HIGH STRESS THRILLER,CREEPY CHILDREN FUCK OFF,OLD PEOPLE,No Theme/Free-for-all
3,10/6/2020 20:56:07,Adventures in Babysitting (1987),River's Edge (1986),Winter's Bone (2010),Girlhood (2014),The 400 Blows (1959),THE NATURAL ENVIRONMENT: FRIEND OR FOE?,CREEPY CHILDREN FUCK OFF,HIGH STRESS THRILLER,THE POWER OF NATURE/THE NATURE OF POWER,OLD PEOPLE,No Theme/Free-for-all
4,10/7/2020 21:30:24,Girlhood (2014),Winter's Bone (2010),The 400 Blows (1959),Adventures in Babysitting (1987),River's Edge (1986),HIGH STRESS THRILLER,THE NATURAL ENVIRONMENT: FRIEND OR FOE?,CREEPY CHILDREN FUCK OFF,OLD PEOPLE,THE POWER OF NATURE/THE NATURE OF POWER,NaN
5,10/8/2020 9:21:17,Winter's Bone (2010),River's Edge (1986),Adventures in Babysitting (1987),The 400 Blows (1959),Girlhood (2014),THE POWER OF NATURE/THE NATURE OF POWER,THE NATURAL ENVIRONMENT: FRIEND OR FOE?,OLD PEOPLE,HIGH STRESS THRILLER,CREEPY CHILDREN FUCK OFF,No Theme/Free-for-all
6,10/8/2020 17:05:25,Adventures in Babysitting (1987),Winter's Bone (2010),Girlhood (2014),River's Edge (1986),NaN,HIGH STRESS THRILLER,THE POWER OF NATURE/THE NATURE OF POWER,THE NATURAL ENVIRONMENT: FRIEND OR FOE?,OLD PEOPLE,CREEPY CHILDREN FUCK OFF,NaN
7,10/8/2020 17:07:14,Girlhood (2014),Adventures in Babysitting (1987),Winter's Bone (2010),River's Edge (1986),The 400 Blows (1959),OLD PEOPLE,THE POWER OF NATURE/THE NATURE OF POWER,THE NATURAL ENVIRONMENT: FRIEND OR FOE?,CREEPY CHILDREN FUCK OFF,HIGH STRESS THRILLER,No Theme/Free-for-all


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(["River's Edge (1986)", 'The 400 Blows (1959)',
       "Winter's Bone (2010)", 'Adventures in Babysitting (1987)',
       'Girlhood (2014)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{"River's Edge (1986)": <Candidate('River's Edge (1986)')>,
 'The 400 Blows (1959)': <Candidate('The 400 Blows (1959)')>,
 "Winter's Bone (2010)": <Candidate('Winter's Bone (2010)')>,
 'Adventures in Babysitting (1987)': <Candidate('Adventures in Babysitting (1987)')>,
 'Girlhood (2014)': <Candidate('Girlhood (2014)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(The 400 Blows (1959), Girlhood (2014), River's Edge (1986), Adventures in Babysitting (1987))>,
 <Ballot(River's Edge (1986), The 400 Blows (1959), Winter's Bone (2010), Adventures in Babysitting (1987), Girlhood (2014))>,
 <Ballot(Winter's Bone (2010), River's Edge (1986), The 400 Blows (1959), Adventures in Babysitting (1987), Girlhood (2014))>,
 <Ballot(Adventures in Babysitting (1987), River's Edge (1986), Winter's Bone (2010), Girlhood (2014), The 400 Blows (1959))>,
 <Ballot(Girlhood (2014), Winter's Bone (2010), The 400 Blows (1959), Adventures in Babysitting (1987), River's Edge (1986))>,
 <Ballot(Winter's Bone (2010), River's Edge (1986), Adventures in Babysitting (1987), The 400 Blows (1959), Girlhood (2014))>,
 <Ballot(Adventures in Babysitting (1987), Winter's Bone (2010), Girlhood (2014), River's Edge (1986))>,
 <Ballot(Girlhood (2014), Adventures in Babysitting (1987), Winter's Bone (2010), River's Edge (1986), The 400 Blows (1959))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                           Votes  Status
--------------------------------  -------  --------
Winter's Bone (2010)                    2  Hopeful
Adventures in Babysitting (1987)        2  Hopeful
Girlhood (2014)                         2  Hopeful
River's Edge (1986)                     1  Rejected
The 400 Blows (1959)                    1  Rejected

ROUND 2
Candidate                           Votes  Status
--------------------------------  -------  --------
Winter's Bone (2010)                    3  Hopeful
Girlhood (2014)                         3  Hopeful
Adventures in Babysitting (1987)        2  Rejected
River's Edge (1986)                     0  Rejected
The 400 Blows (1959)                    0  Rejected

FINAL RESULT
Candidate                           Votes  Status
--------------------------------  -------  --------
Winter's Bone (2010)                    5  Elected
Girlhood (2014)                         3  Rejected
Adventures in Babysitting (1987)        0  Re

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                  Votes  Status
---------------------------------------  -------  --------
THE POWER OF NATURE/THE NATURE OF POWER        3  Hopeful
HIGH STRESS THRILLER                           2  Hopeful
THE NATURAL ENVIRONMENT: FRIEND OR FOE?        1  Hopeful
CREEPY CHILDREN FUCK OFF                       1  Hopeful
OLD PEOPLE                                     1  Rejected
No Theme/Free-for-all                          0  Rejected

FINAL RESULT
Candidate                                  Votes  Status
---------------------------------------  -------  --------
THE POWER OF NATURE/THE NATURE OF POWER        4  Elected
HIGH STRESS THRILLER                           2  Rejected
THE NATURAL ENVIRONMENT: FRIEND OR FOE?        1  Rejected
CREEPY CHILDREN FUCK OFF                       1  Rejected
OLD PEOPLE                                     0  Rejected
No Theme/Free-for-all                          0  Rejected

